In [1]:
source(file.path(Sys.getenv("CODEBASE"),"DNAmeth500species/src/00.0_init.R"))

Warning message:
“package ‘ggrepel’ was built under R version 3.6.3”
Warning message:
“package ‘ggseqlogo’ was built under R version 3.6.3”
Warning message:
“package ‘pheatmap’ was built under R version 3.6.3”
Joining, by = "species"



In [2]:
library(GEOquery)

Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Setting options('download.file.method.GEOquery'='auto')

Setting options('GEOquery.inmemory.gpl'=FALSE)



In [3]:
library(dplyr)
library(data.table)

In [7]:
#Function, that downloads the annotation table based on the gse_query:
get_gse_df <- function(gse_query){
    gse <- getGEO(gse_query)
    gse <- gse[[1]]
    df <- pData(gse)
    
    return(df)
}

In [8]:
##function how to handle 2 bedgraph files (ratio and coverage)
download_coverage_and_ratio_bedgraph <- function(path_cov, path_ratio, dir_to_save, id_to_save){
    ####ratio
    ##download
    cmd = paste0("wget ", path_ratio, " -P ", dir_to_save)
    system(cmd)
    ##unzip
    id <- strsplit(as.character(path_ratio), "/")[[1]]
    id_ratio <- id[length(id)]
    cmd =  paste0("gunzip ", dir_to_save, id_ratio)
    system(cmd)
    ####coverage
    ##download
    cmd = paste0("wget ", path_cov, " -P ", dir_to_save)
    system(cmd)
    ##unzip
    id <- strsplit(as.character(path_cov), "/")[[1]]
    id_cov <- id[length(id)]
    cmd =  paste0("gunzip ", dir_to_save, id_cov)
    system(cmd)
    
    ##merge
    df_cov <- read.table(paste0(dir_to_save, gsub(".gz", "", id_cov)), skip = 1, sep = "\t")
    colnames(df_cov) <- c("chr", "start", "end", "cov")
    
    df_ratio <- read.table(paste0(dir_to_save, gsub(".gz", "", id_ratio)), skip = 1, sep = "\t")
    colnames(df_ratio) <- c("chr", "start", "end", "ratio")
    
    df <- full_join(df_ratio, df_cov)
    write.table(df, paste0(dir_to_save, id_to_save,".tsv"), sep = "\t", row.names = F, quote = F)
    
    print("done")
    
}

In [9]:
download_cpg_report <- function(path, dir_to_save){
     ##download
    cmd = paste0("wget ", path, " -P ", dir_to_save)
    system(cmd)
    ##unzip
    id <- strsplit(as.character(path), "/")[[1]]
    id<- id[length(id)]
    cmd =  paste0("gunzip ", dir_to_save, id)
    system(cmd)
}

In [3]:
WGBS_dir <- file.path(data_dir, "resources","WGBS_public")

In [4]:
WGBS_dir

[1] "/binfl/lv71484/droman/DNAmeth500species//resources/WGBS_public"

In [5]:
setwd(WGBS_dir)

## Gallus gallus time series dataset

In [6]:
df_gallus <- get_gse_df("GSE146620")

Found 1 file(s)

GSE146620_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp5XEq1S/GSE146620_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


Using locally cached version of GPL24517 found here:
/tmp/Rtmp5XEq1S/GPL24517.soft 



In [7]:
head(df_gallus)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,library_source,library_strategy,relation,relation.1,supplementary_file_1,supplementary_file_2,age:ch1,disease state:ch1,tissue:ch1,treatment:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSM4399753,Breast_1_d3,GSM4399753,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333161,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885080,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399753/suppl/GSM4399753_Breast_1_d3_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399753/suppl/GSM4399753_Breast_1_d3_ratio.bedgraph.gz,3 days after birth,healthy,breast tissue,No treatment
GSM4399754,Breast_1_d15,GSM4399754,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333160,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885086,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399754/suppl/GSM4399754_Breast_1_d15_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399754/suppl/GSM4399754_Breast_1_d15_ratio.bedgraph.gz,15 days after birth,healthy,breast tissue,No treatment
GSM4399755,Breast_1_d28,GSM4399755,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333159,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885087,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399755/suppl/GSM4399755_Breast_1_d28_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399755/suppl/GSM4399755_Breast_1_d28_ratio.bedgraph.gz,28 days after birth,healthy,breast tissue,No treatment
GSM4399756,Breast_1_d34,GSM4399756,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333158,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885088,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399756/suppl/GSM4399756_Breast_1_d34_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399756/suppl/GSM4399756_Breast_1_d34_ratio.bedgraph.gz,34 days after birth,healthy,breast tissue,No treatment
GSM4399757,Breast_2_d3,GSM4399757,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333157,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885089,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399757/suppl/GSM4399757_Breast_2_d3_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399757/suppl/GSM4399757_Breast_2_d3_ratio.bedgraph.gz,3 days after birth,healthy,breast tissue,No treatment
GSM4399758,Breast_2_d15,GSM4399758,Public on Nov 30 2020,Mar 09 2020,Dec 01 2020,SRA,1,breast tissue,Gallus gallus,treatment: No treatment,⋯,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14333156,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7885090,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399758/suppl/GSM4399758_Breast_2_d15_cov.bedgraph.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4399nnn/GSM4399758/suppl/GSM4399758_Breast_2_d15_ratio.bedgraph.gz,15 days after birth,healthy,breast tissue,No treatment


in this case we have to prefilter the table, as we don't need most of the samples

In [8]:
unique(df_gallus$source_name_ch1)

[1] breast tissue  ileum tissue   jejunum tissue spleen tissue 
Levels: breast tissue ileum tissue jejunum tissue spleen tissue

In [10]:
unique(df_gallus$`disease state:ch1`)

[1] "healthy"  "inflamed"

In [12]:
df_gallus <- df_gallus[df_gallus$`disease state:ch1`=='healthy',]
df_gallus$age_days <- sapply(df_gallus$`age:ch1`, function(x) strsplit(x, " ")[[1]][1])

In [16]:
df_gallus$age_days <- as.numeric(df_gallus$age_days)

In [19]:
df_gallus <- df_gallus  %>% group_by(source_name_ch1) %>% filter(age_days == max(age_days))

In [69]:
dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df_gallus[2, ]$organism_ch1), "/", gsub(" ", "_",df_gallus[2, ]$source_name_ch1), "/")

In [70]:
dir.create(dir_to_save, recursive = T)

Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Gallus_gallus/breast_tissue' already exists”


In [72]:
id_to_save <-  df_gallus[2, ]$title

In [76]:
download_coverage_and_ratio_bedgraph(df_gallus[2, ]$supplementary_file_1, df_gallus[2, ]$supplementary_file_2, dir_to_save, id_to_save)

Joining, by = c("chr", "start", "end")



[1] "done"


In [78]:
for (i in c(3:NROW(df_gallus))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df_gallus[i, ]$organism_ch1), "/", gsub(" ", "_",df_gallus[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    id_to_save <-  df_gallus[i, ]$title
    print(id_to_save)
    download_coverage_and_ratio_bedgraph(df_gallus[i, ]$supplementary_file_1, df_gallus[i, ]$supplementary_file_2, dir_to_save, id_to_save)  
} 

Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Gallus_gallus/breast_tissue' already exists”


         V12 
Breast_3_d34 
51 Levels: Breast_1_d15 Breast_1_d28 Breast_1_d3 Breast_1_d34 ... Spleen_3_d34


Joining, by = c("chr", "start", "end")



[1] "done"
      V19 
Ile_1_d34 
51 Levels: Breast_1_d15 Breast_1_d28 Breast_1_d3 Breast_1_d34 ... Spleen_3_d34


Joining, by = c("chr", "start", "end")



...

[1] "done"


Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Gallus_gallus/spleen_tissue' already exists”


         V52 
Spleen_3_d34 
51 Levels: Breast_1_d15 Breast_1_d28 Breast_1_d3 Breast_1_d34 ... Spleen_3_d34


Joining, by = c("chr", "start", "end")



[1] "done"


## Cow

In [79]:
df_cow <- get_gse_df("GSE147087")

Found 1 file(s)

GSE147087_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


File stored at: 

/tmp/Rtmp5XEq1S/GPL24230.soft



In [80]:
df_cow

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,library_selection,library_source,library_strategy,relation,relation.1,relation.2,supplementary_file_1,breed:ch1,Sex:ch1,tissue:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>
GSM4238269,"Holstein, Adipose_1_WGBS",GSM4238269,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Adipose,Bos taurus,tissue: Adipose,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699103,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001109,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478295,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238269/suppl/GSM4238269_Adipose_3842_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Adipose
GSM4238270,"Holstein, Adipose_2_WGBS",GSM4238270,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Adipose,Bos taurus,tissue: Adipose,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699102,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001110,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478296,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238270/suppl/GSM4238270_Adipose_3886_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Adipose
GSM4238271,"Holstein, Blood_1_WGBS",GSM4238271,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Blood,Bos taurus,tissue: Blood,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699101,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001111,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478297,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238271/suppl/GSM4238271_Blood_4091_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Blood
GSM4238272,"Holstein, Blood_2_WGBS",GSM4238272,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Blood,Bos taurus,tissue: Blood,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699100,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001112,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478298,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238272/suppl/GSM4238272_Blood_4254_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Blood
GSM4238273,"Holstein, Heart_1_WGBS",GSM4238273,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Heart,Bos taurus,tissue: Heart,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699099,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001113,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478299,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238273/suppl/GSM4238273_Heart_3842_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Heart
GSM4238274,"Holstein, Ileum_1_WGBS",GSM4238274,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Ileum,Bos taurus,tissue: Ileum,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699098,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001114,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478300,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238274/suppl/GSM4238274_Ileum_3842_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Ileum
GSM4238275,"Holstein, Kidney_1_WGBS",GSM4238275,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Kidney,Bos taurus,tissue: Kidney,⋯,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13699097,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8001115,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7478301,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4238nnn/GSM4238275/suppl/GSM4238275_Kidney_3842_1.fq.gz_bismark_bt2_pe.deduplicated.CpG_report.txt.gz,Holstein,F,Kidney
GSM4238276,"Holstein, Ldorsi_1_WGBS",GSM4238276,Public on Apr 30 2020,Dec 30 2019,Apr 30 2020,SRA,1,Ldorsi,Bos taurus,tissue: Ldorsi,⋯,RANDOM,geno

In [83]:
NROW(df_cow)

[1] 25

In [82]:
for (i in c(1:NROW(df_cow))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df_cow[i, ]$organism_ch1), "/", gsub(" ", "_",df_cow[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    download_cpg_report(df_cow[i, ]$supplementary_file_1, dir_to_save)  
} 

Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Adipose' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Blood' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Ldorsi' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Lung' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Rumen' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Placenta' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Adipose' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Bos_taurus/Heart' already exists”
Warning m

## Mouse

In [84]:
df <- get_gse_df("GSE42836")
df

Found 1 file(s)

GSE42836_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM1051150 = col_character(),
  GSM1051151 = col_character(),
  GSM1051152 = col_character(),
  GSM1051153 = col_character(),
  GSM1051154 = col_character(),
  GSM1051155 = col_character(),
  GSM1051156 = col_character(),
  GSM1051157 = col_character(),
  GSM1051158 = col_character(),
  GSM1051159 = col_character(),
  GSM1051160 = col_character(),
  GSM1051161 = col_character(),
  GSM1051162 = col_character(),
  GSM1051163 = col_character(),
  GSM1051164 = col_character(),
  GSM1051165 = col_character(),
  GSM1051166 = col_character()
)


File stored at: 

/tmp/Rtmp5XEq1S/GPL13112.soft



,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,instrument_model,library_selection,library_source,library_strategy,relation,relation.1,supplementary_file_1,developmental stage:ch1,strain:ch1,tissue:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>
GSM1051150,mouse bone marrow,GSM1051150,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse bone marrow,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209448,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830647,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051150/suppl/GSM1051150_bone_marrow.CpG.calls.txt.gz,adult,C57Bl/6,bone marrow
GSM1051151,mouse cerebellum,GSM1051151,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse cerebellum,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209449,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830648,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051151/suppl/GSM1051151_cerebellum.CpG.calls.txt.gz,adult,C57Bl/6,cerebellum
GSM1051152,mouse colon,GSM1051152,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse colon,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209450,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830649,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051152/suppl/GSM1051152_colon.CpG.calls.txt.gz,adult,C57Bl/6,colon
GSM1051153,mouse cortex,GSM1051153,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse cortex,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209451,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830650,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051153/suppl/GSM1051153_cortex.CpG.calls.txt.gz,adult,C57Bl/6,cortex
GSM1051154,mouse heart,GSM1051154,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse heart,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209452,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830651,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051154/suppl/GSM1051154_heart.CpG.calls.txt.gz,adult,C57Bl/6,heart
GSM1051155,mouse intestine,GSM1051155,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse intestine,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209453,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830652,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051155/suppl/GSM1051155_intestine.CpG.calls.txt.gz,adult,C57Bl/6,intestine
GSM1051156,mouse kidney,GSM1051156,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse kidney,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209454,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830653,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051156/suppl/GSM1051156_kidney.CpG.calls.txt.gz,adult,C57Bl/6,kidney
GSM1051157,mouse liver,GSM1051157,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse liver,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX209455,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01830654,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1051nnn/GSM1051157/suppl/GSM1051157_liver.CpG.calls.txt.gz,adult,C57Bl/6,liver
GSM1051158,mouse lung,GSM1051158,Public on Sep 01 2013,Dec 10 2012,May 15 2019,SRA,1,mouse lung,Mus musculus,strain: C57Bl/6,⋯,Illumina HiSeq 2000,RANDOM,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.ni

In [86]:
for (i in c(1:NROW(df))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df[i, ]$organism_ch1), "/", gsub(" ", "_",df[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    download_cpg_report(df[i, ]$supplementary_file_1, dir_to_save)  
} 

## Lancet 1

In [88]:
df <- get_gse_df("GSE102144")

Found 2 file(s)

GSE102144-GPL23852_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM2728827 = col_character(),
  GSM2728828 = col_character(),
  GSM2728829 = col_character(),
  GSM2728830 = col_character()
)


File stored at: 

/tmp/Rtmp5XEq1S/GPL23852.soft

GSE102144-GPL25186_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM3190302 = col_character(),
  GSM3190303 = col_character(),
  GSM3190304 = col_character(),
  GSM3190305 = col_character(),
  GSM3190306 = col_character(),
  GSM3190307 = col_character()
)


File stored at: 

/tmp/Rtmp5XEq1S/GPL25186.soft



In [90]:
df <- df[df["developmental stage:ch1"] == "adult",]

In [94]:
dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df[1, ]$organism_ch1),"_1", "/", gsub(" ", "_",df[1, ]$source_name_ch1), "/")

dir.create(dir_to_save, recursive = T)

download_cpg_report(df[1, ]$supplementary_file_1, dir_to_save)   

## Multi-species study 
a bit different, as there is more than 

In [10]:
 gse <- getGEO("GSE141609")

Found 9 file(s)

GSE141609-GPL13725_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmpg8tL1S/GSE141609-GPL13725_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM4209497 = col_character(),
  GSM4209501 = col_character()
)


Using locally cached version of GPL13725 found here:
/tmp/Rtmpg8tL1S/GPL13725.soft 

GSE141609-GPL18171_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmpg8tL1S/GSE141609-GPL18171_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM4209494 = col_character()
)


Using locally cached version of GPL18171 found here:
/tmp/Rtmpg8tL1S/GPL18171.soft 

GSE141609-GPL21476_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmpg8tL1S/GSE141609-GPL21476_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_RE

In [14]:
length(gse)

[1] 9

In [17]:
df = data.table()
for (i in c(1:9)){
    df_temp <- pData(gse[[i]])
    df <- rbind(df, df_temp)
}

In [ ]:

    gse <- gse[[1]]
    df <- pData(gse)

In [19]:
df <- df[df$library_strategy == "Bisulfite-Seq",]

In [20]:
df

title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,relation.1,supplementary_file_1,source:ch1
<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
mC_brain_queen_honeybee,GSM4209497,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,brain,Apis mellifera,"source: Reared at UWA, Australia",⋯,Australia,0,Illumina Genome Analyzer IIx,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508509,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285996,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209497/suppl/GSM4209497_mC_brain_queen_honeybee.CGmap.gz,"Reared at UWA, Australia"
mC_forebrain_zebrafish,GSM4209494,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,forebrain,Danio rerio,source: Mix AB/Tubingen,⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508512,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285993,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209494/suppl/GSM4209494_mC_forebrain_zebrafish.CGmap.gz,Mix AB/Tubingen
mC_brain_chicken,GSM4209493,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,brain,Gallus gallus,"source: Collected in Seville, Spain",⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508513,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285992,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209493/suppl/GSM4209493_mC_brain_chicken.CGmap.gz,"Collected in Seville, Spain"
mC_brain_opossum,GSM4209491,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,brain,Monodelphis domestica,source: Frozen sample,⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508515,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285990,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209491/suppl/GSM4209491_mC_brain_opossum.CGmap.gz,Frozen sample
mC_brain_platypus,GSM4209492,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,frontal cortex,Ornithorhynchus anatinus,source: Frozen sample,⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508514,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285991,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209492/suppl/GSM4209492_mC_brain_platypus.CGmap.gz,Frozen sample
mC_brain_elephant_shark,GSM4209495,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,brain,Callorhinchus milii,source: Frozen sample,⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508511,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285994,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209495/suppl/GSM4209495_mC_brain_elephant_shark.CGmap.gz,Frozen sample
mC_brain_arctic_lamprey,GSM4209496,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,brain,Lethenteron camtschaticum,source: Frozen sample,⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508510,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285995,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209496/suppl/GSM4209496_mC_brain_arctic_lamprey.CGmap.gz,Frozen sample
mC_brain_octopus_supraesophageal,GSM4209498,Public on Oct 26 2020,Dec 06 2019,Oct 27 2020,SRA,1,Supraesophageal brain,Octopus bimaculoides,"source: Ragsdale laboratory, Chicago",⋯,Australia,0,Illumina HiSeq 1500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN13508508,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX7285997,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4209nnn/GSM4209498/suppl/GSM4209498_mC_brain_octopus_supraesophageal.CGmap.gz,"Rag

In [21]:
for (i in c(1:NROW(df))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df[i, ]$organism_ch1), "/", gsub(" ", "_",df[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    download_cpg_report(df[i, ]$supplementary_file_1, dir_to_save)  
} 

### big koala dataset

In [22]:
df <- get_gse_df("GSE149600")

Found 1 file(s)

GSE149600_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM4505500 = col_character(),
  GSM4505501 = col_character(),
  GSM4505502 = col_character(),
  GSM4505503 = col_character(),
  GSM4505504 = col_character(),
  GSM4505505 = col_character(),
  GSM4505506 = col_character(),
  GSM4505507 = col_character(),
  GSM4505508 = col_character(),
  GSM4505509 = col_character()
)


File stored at: 

/tmp/Rtmpg8tL1S/GPL26281.soft



In [23]:
df

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,instrument_model,library_selection,library_source,library_strategy,relation,relation.1,supplementary_file_1,australian museum registration:ch1,Sex:ch1,tissue:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>
GSM4505500,koala_WGBS_PC_brain,GSM4505500,Public on Feb 27 2021,Apr 29 2020,Mar 01 2021,SRA,1,Right hemisphere cerebral lobe,Phascolarctos cinereus,Sex: female,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771362,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207654,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505500/suppl/GSM4505500_WGM145_01_S1_brain_PC_CpG_report.txt.gz,M.45022,female,brain
GSM4505501,koala_WGBS_Ben_brain,GSM4505501,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,Right hemisphere cerebral lobe,Phascolarctos cinereus,Sex: male,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771360,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207655,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505501/suppl/GSM4505501_WGM145_04_S2_brain_Ben_CpG_report.txt.gz,M.47723,male,brain
GSM4505502,koala_WGBS_PC_kidney,GSM4505502,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,kidney,Phascolarctos cinereus,Sex: female,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771359,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207656,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505502/suppl/GSM4505502_WGM145_06_S3_kidney_PC_CpG_report.txt.gz,M.45022,female,kidney
GSM4505503,koala_WGBS_Ben_kidney,GSM4505503,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,kidney,Phascolarctos cinereus,Sex: male,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771357,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207657,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505503/suppl/GSM4505503_WGM145_08_S4_kidney_Ben_CpG_report.txt.gz,M.47723,male,kidney
GSM4505504,koala_WGBS_PC_lung,GSM4505504,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,lung,Phascolarctos cinereus,Sex: female,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771355,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207658,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505504/suppl/GSM4505504_WGM145_09_S5_lung_PC_CpG_report.txt.gz,M.45022,female,lung
GSM4505505,koala_WGBS_Ben_lung,GSM4505505,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,lung,Phascolarctos cinereus,Sex: male,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771353,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207659,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505505/suppl/GSM4505505_WGM145_12_S6_lung_Ben_CpG_report.txt.gz,M.47723,male,lung
GSM4505506,koala_WGBS_PC_skeletal_muscle,GSM4505506,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,skeletal muscle,Phascolarctos cinereus,Sex: female,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771351,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207660,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505506/suppl/GSM4505506_WGM145_14_S7_skeletal_muscle_PC_CpG_report.txt.gz,M.45022,female,skeletal muscle
GSM4505507,koala_WGBS_Ben_skeletal_muscle,GSM4505507,Public on Feb 27 2021,Apr 29 2020,Mar 02 2021,SRA,1,skeletal muscle,Phascolarctos cinereus,Sex: male,⋯,Illumina NovaSeq 6000,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN14771349,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX8207661,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4505nnn/GSM4505507/suppl/G

In [25]:
for (i in c(1:NROW(df))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df[i, ]$organism_ch1), "/", gsub(" ", "_",df[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    download_cpg_report(df[i, ]$supplementary_file_1, dir_to_save)  
} 

Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Phascolarctos_cinereus/Right_hemisphere_cerebral_lobe' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Phascolarctos_cinereus/kidney' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Phascolarctos_cinereus/lung' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Phascolarctos_cinereus/skeletal_muscle' already exists”
Warning message in dir.create(dir_to_save, recursive = T):
“'../../resources/WGBS_public/Phascolarctos_cinereus/pancreas' already exists”


## oyster

In [26]:
df <- get_gse_df("GSE40302")

Found 1 file(s)

GSE40302_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM991064 = col_character(),
  GSM991065 = col_character()
)


File stored at: 

/tmp/Rtmpg8tL1S/GPL13986.soft



In [27]:
df

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,library_source,library_strategy,relation,relation.1,relation.2,supplementary_file_1,gender:ch1,individual type:ch1,sample type:ch1,tissue:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSM991064,Inbred,GSM991064,Public on Dec 18 2014,Aug 22 2012,May 15 2019,SRA,1,mantle of the inbred individual,Crassostrea gigas,individual type: inbred,⋯,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX180559,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX180560,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01125790,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM991nnn/GSM991064/suppl/GSM991064_Inbred.Cdepth.txt.gz,female,inbred,reference,mantle
GSM991065,Wild,GSM991065,Public on Dec 18 2014,Aug 22 2012,May 15 2019,SRA,1,mantle of the wild individual,Crassostrea gigas,individual type: wild,⋯,genomic,Bisulfite-Seq,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX180561,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX180562,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN01125791,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM991nnn/GSM991065/suppl/GSM991065_Wild.Cdepth.txt.gz,NA,wild,NA,mantle


In [28]:
for (i in c(1:NROW(df))){
    dir_to_save = paste0("../../resources/WGBS_public/", gsub(" ", "_", df[i, ]$organism_ch1), "/", df[i, "tissue:ch1"],"_", gsub(" ", "_",df[i, ]$source_name_ch1), "/")
    dir.create(dir_to_save, recursive = T)
    download_cpg_report(df[i, ]$supplementary_file_1, dir_to_save)  
} 

## Xenopus

In [ ]:
#sample 1
df <- get_gse_df("GSE90898")

In [ ]:
download_cpg_report(df[1,]$supplementary_file_1, "../../resources/WGBS_public/Xenopus_laevis/")

In [ ]:
#sample 2
df <- get_gse_df("GSE76247") # error

In [ ]:
gse <- getGEO("GSM1977638")

In [ ]:
download_cpg_report(gse@header$supplementary_file_1, "../../resources/WGBS_public/Xenopus_laevis/")

## Raw file download (using SRA)

### zebrafish 1

In [12]:
df <- get_gse_df("GSE149416")

Found 1 file(s)

GSE149416_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp9qwIgW/GSE149416_series_matrix.txt.gz

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


Using locally cached version of GPL23085 found here:
/tmp/Rtmp9qwIgW/GPL23085.soft 



ERROR: Error in `sampleNames<-`(`*tmp*`, value = sampleNames(phenoData)): 'value' length (38) must equal sample number in AssayData (0)


annotation - made by hand
download - from SRA

In [13]:
### Danio reiro 2

In [16]:
 gse <- getGEO("GSE134055")

Found 4 file(s)

GSE134055-GPL10164_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp9qwIgW/GSE134055-GPL10164_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


Using locally cached version of GPL10164 found here:
/tmp/Rtmp9qwIgW/GPL10164.soft 

GSE134055-GPL14875_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp9qwIgW/GSE134055-GPL14875_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM4662087 = col_character(),
  GSM4662088 = col_character()
)


Using locally cached version of GPL14875 found here:
/tmp/Rtmp9qwIgW/GPL14875.soft 

GSE134055-GPL23085_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp9qwIgW/GSE134055-GPL23085_series_matrix.txt.gz


── Column specification ─────────────────────────────────────────────────

In [20]:
df = data.table()
for (i in c(1:length(gse))){
    df_temp <- pData(gse[[i]])
    df <- rbind(df, df_temp, fill = T)
}

In [57]:
head(df)

title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,extract_protocol_ch1.8,extract_protocol_ch1.9,extract_protocol_ch1.10,extract_protocol_ch1.11,description.1,description.2,description.3,replica,GSE,SRR
<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<chr>,<chr>
YueLab-Blood-WGBS,GSM4662070,Public on Sep 04 2020,Jul 08 2020,Sep 05 2020,SRA,1,YueLab-Blood-WGBS,Danio rerio,strain: Tuebingen,⋯,"WGBS: Tübingen adult tissues were dissected, and the genomic DNA was extracted using DNeasy Blood & Tissue Kit (Qiagen, 69504). Then 1 µg of genomic DNA of each adult tissue was subjected to bisulfite conversion using EZ DNA Methylation-Gold Kits",WGBS: The final libraries were prepared using the Accel-NGS® Methyl-Seq DNA Library Kit.,"scATAC-seq: Single-cell ATAC-seq (scATAC-seq) was performed in one female brain and one male brain, respectively, on the 10X Genomics platform. To isolate nuclei, freshly dissected single brain was transferred to 1 mL NbActiv1 medium (BrainBits, NbActiv1 500) with a wide-bore pipette tip to break the tissue into small pieces. Then the tissue was broken up completely using regular-bore pipette tips followed by filtering through a 30 µm cell strainer. The isolated cells were spun down at 500 g for 5 min at 4 °C and then lysed in 100 µL chilled 0.1X Lysis Buffer (10 mM Tris HCl, pH 7.5, 10 mM NaCl, 3 mM MgCl2, 1% BSA,0.01% Tween-20, 0.01% NP40 and 0.001% digitonin) on ice for 5 min. Then 1 mL chilled Wash Buffer (10 mM Tris HCl, pH 7.5, 10 mM NaCl, 3 mM MgCl2, 1% BSA, 0.1% Tween-20) was added to the lysed cells and cells were spun down at 500 g for 5 min at 4 °C. At last, 300 µL chilled Diluted Nuclei Buffer (10x Genomics, PN 2000153/2000207) was added to resuspend the nuclei. The nuclei were filtered again using a 30 µm cell strainer before cell counting","scATAC-seq: Around 12,000 nuclei were used for one Tn5 tagmentation reaction, and the scATAC-seq library was prepared and sequenced exactly according to the company’s user guide.",,,NA,1,GSE134055,SRX8688317
YueLab-Brain-WGBS,GSM4662071,Public on Sep 04 2020,Jul 08 2020,Sep 05 2020,SRA,1,YueLab-Brain-WGBS,Danio rerio,strain: Tuebingen,⋯,"WGBS: Tübingen adult tissues were dissected, and the genomic DNA was extracted using DNeasy Blood & Tissue Kit (Qiagen, 69504). Then 1 µg of genomic DNA of each adult tissue was subjected to bisulfite conversion using EZ DNA Methylation-Gold Kits",WGBS: The final libraries were prepared using the Accel-NGS® Methyl-Seq DNA Library Kit.,"scATAC-seq: Single-cell ATAC-seq (scATAC-seq) was performed in one female brain and one male brain, respectively, on the 10X Genomics platform. To isolate nuclei, freshly dissected single brain was transferred to 1 mL NbActiv1 medium (BrainBits, NbActiv1 500) with a wide-bore pipette tip to break the tissue into small pieces. Then the tissue was broken up completely using regular-bore pipette tips followed by filtering through a 30 µm cell strainer. The isolated cells were spun down at 500 g for 5 min at 4 °C and then lysed in 100 µL chilled 0.1X Lysis Buffer (10 mM Tris HCl, pH 7.5, 10 mM NaCl, 3 mM MgCl2, 1% BSA,0.01% Tween-20, 0.01% NP40 and 0.001% digitonin) on ice for 5 min. Then 1 mL chilled Wash Buffer (10 mM Tris HCl, pH 7.5, 10 mM NaCl, 3 mM MgCl2, 1% BSA, 0.1% Tween-20) was added to the lysed cells and cells were spun down at 500 g for 5 min at 4 °C. At last, 300 µL chilled Diluted Nuclei Buffer (10x Genomics, PN 2000153/2000207) was added to resuspend the nuclei. The nuclei were filtered again using a 30 µm cell strainer before cell counting","scATAC-seq: Around 12,000 nuclei were used for one Tn5 tagmentation reaction, and the scATAC-seq library was prepared and sequenced exactly according to the company’s user guide.",,,NA,1,GSE134055,SRX8688318
YueLab-Colon-WGBS,GSM4662072,Public on Sep 04 2020,Jul 08 2020,Sep 05 2020,SRA,1,YueLab-Colon-WGBS

In [25]:
df <- df[sapply(df$title, function(x) length(grep("WGBS", x))>0),]

In [35]:
colnames(df)[48] <- "tissue"

In [38]:
df$replica <- 1

In [39]:
df$GSE <- "GSE134055"

In [52]:
df$SRR <- sapply(df$`relation.1`, function(x) strsplit(as.character(x), '=')[[1]][2])

In [56]:
write.table(df[, c("geo_accession", "tissue", "replica", "GSE", "SRR")],
            "../../resources/WGBS_public/Danio_rerio_GSE134055/GSE134055_WGBS_adult.csv", quote = F, row.names = F, sep=";")

## Download corresponding genomes

In [8]:
library(BSgenome)

In [14]:
av_gen <- available.genomes(splitNameParts=TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org




In [15]:
unique(av_gen$organism)

[1] Alyrata       Amellifera    Aofficinalis  Athaliana     Btaurus      
 [6] Carietinum    Celegans      Cfamiliaris   Cjacchus      Dmelanogaster
[11] Drerio        Ecoli         Gaculeatus    Ggallus       Hsapiens     
[16] Mdomestica    Mfascicularis Mfuro         Mmulatta      Mmusculus    
[21] Osativa       Ptroglodytes  Rnorvegicus   Scerevisiae   Sscrofa      
[26] Tgondii       Tguttata      Vvinifera    
28 Levels: Alyrata Amellifera Aofficinalis Athaliana Btaurus ... Vvinifera

In [18]:
av_gen[av_gen$provider_version=="braLan2",]

pkgname,organism,provider,provider_version,masked
<chr>,<fct>,<fct>,<chr>,<lgl>


In [6]:
av_gen[av_gen$organism == "Mmusculus",]

,pkgname,organism,provider,provider_version,masked
,<chr>,<fct>,<fct>,<chr>,<lgl>
67,BSgenome.Mmusculus.UCSC.mm10,Mmusculus,UCSC,mm10,FALSE
68,BSgenome.Mmusculus.UCSC.mm10.masked,Mmusculus,UCSC,mm10,TRUE
69,BSgenome.Mmusculus.UCSC.mm8,Mmusculus,UCSC,mm8,FALSE
70,BSgenome.Mmusculus.UCSC.mm8.masked,Mmusculus,UCSC,mm8,TRUE
71,BSgenome.Mmusculus.UCSC.mm9,Mmusculus,UCSC,mm9,FALSE
72,BSgenome.Mmusculus.UCSC.mm9.masked,Mmusculus,UCSC,mm9,TRUE


In [8]:
av_gen[av_gen$organism == "Ggallus",]

,pkgname,organism,provider,provider_version,masked
,<chr>,<fct>,<fct>,<chr>,<lgl>
42,BSgenome.Ggallus.UCSC.galGal3,Ggallus,UCSC,galGal3,FALSE
43,BSgenome.Ggallus.UCSC.galGal3.masked,Ggallus,UCSC,galGal3,TRUE
44,BSgenome.Ggallus.UCSC.galGal4,Ggallus,UCSC,galGal4,FALSE
45,BSgenome.Ggallus.UCSC.galGal4.masked,Ggallus,UCSC,galGal4,TRUE
46,BSgenome.Ggallus.UCSC.galGal5,Ggallus,UCSC,galGal5,FALSE
47,BSgenome.Ggallus.UCSC.galGal6,Ggallus,UCSC,galGal6,FALSE


In [10]:
genomes_avail <- ucscGenomes()

In [11]:
head(genomes_avail)

,db,species,date,name
,<fct>,<fct>,<fct>,<fct>
1,hg38,Human,Dec. 2013,Genome Reference Consortium GRCh38
2,hg19,Human,Feb. 2009,Genome Reference Consortium GRCh37
3,hg18,Human,Mar. 2006,NCBI Build 36.1
4,hg17,Human,May 2004,NCBI Build 35
5,hg16,Human,Jul. 2003,NCBI Build 34
6,vicPac2,Alpaca,Mar. 2013,Broad Institute Vicugna_pacos-2.0.1


In [12]:
unique(genomes_avail$species)

[1] Human                           Alpaca                         
  [3] Armadillo                       Baboon                         
  [5] Bison                           Bonobo                         
  [7] Brown kiwi                      Bushbaby                       
  [9] Cat                             Chimp                          
 [11] Chinese hamster                 Chinese hamster ovary cell line
 [13] Chinese pangolin                Cow                            
 [15] Crab-eating macaque             Dog                            
 [17] Dolphin                         Elephant                       
 [19] Ferret                          Garter snake                   
 [21] Gibbon                          Golden eagle                   
 [23] Golden snub-nosed monkey        Gorilla                        
 [25] Green Monkey                    Guinea pig                     
 [27] Hawaiian monk seal              Hedgehog                       
 [29] Horse                           Kangaroo rat                   
 [31] Malayan flying lemur            Manatee                        
 [33] Marmoset                        Megabat                        
 [35] Microbat                        Minke whale                    
 [37] Mouse                           Mouse lemur                    
 [39] Naked mole-rat                  Opossum                        
 [41] Orangutan                       Panda                          
 [43] Pig                             Pika                           
 [45] Platypus                        Proboscis Monkey               
 [47] Rabbit                          Rat                            
 [49] Rhesus                          Rock hyrax                     
 [51] Sheep                           Shrew                          
 [53] Sloth                           Squirrel                       
 [55] Squirrel monkey                 Tarsier                        
 [57] Tasmanian devil                 Tenrec                         
 [59] Tree shrew                      Wallaby                        
 [61] White rhinoceros                African clawed frog            
 [63] American alligator              Atlantic cod                   
 [65] Budgerigar                      Chicken                        
 [67] Coelacanth                      Elephant shark                 
 [69] Fugu                            Lamprey                        
 [71] Lizard                          Medaka                         
 [73] Medium ground finch             Nile tilapia                   
 [75] Painted turtle                  Stickleback                    
 [77] Tetraodon                       Tibetan frog                   
 [79] Turkey                          X. tropicalis                  
 [81] Zebra finch                     Zebrafish                      
 [83] C. intestinalis                 Lancelet                       
 [85] S. purpuratus                   A. mellifera                   
 [87] A. gambiae                      D. ananassae                   
 [89] D. erecta                       D. grimshawi                   
 [91] D. melanogaster                 D. mojavensis                  
 [93] D. persimilis                   D. pseudoobscura               
 [95] D. sechellia                    D. simulans                    
 [97] D. virilis                      D. yakuba                      
 [99] C. brenneri                     C. briggsae                    
[101] C. elegans                      C. japonica                    
[103] C. remanei                      P. pacificus                   
[105] Sea Hare                        Yeast                          
[107] Ebola Virus                     SARS-CoV-2                     
116 Levels:  A. gambiae A. mellifera African clawed frog ... Zebrafish

In [6]:
library(BSgenome)

Loading required package: rtracklayer



## Install missing genomes for the WGBS: preparing BSgenome packages and downloading fastas

## Mmusculus:
mm9

In [1]:
BiocManager::install("BSgenome.Mmusculus.UCSC.mm9")

Bioconductor version '3.10' is out-of-date; the current release version '3.12'
  is available with R version '4.0'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.12), R 3.6.1 (2019-07-05)

Installing package(s) 'BSgenome.Mmusculus.UCSC.mm9'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocManager', 'devtools', 'dplyr', 'IRkernel', 'stringi'



In [1]:
library(BSgenome.Mmusculus.UCSC.mm9)

Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4




In [2]:
genome <- get("Mmusculus")

In [3]:
dir.create("../../resources/WGBS_public/Mus_musculus/refgenome")

In [6]:
export(genome, "../../resources/WGBS_public/Mus_musculus/refgenome/mm9.fasta")

writing chr1 sequence to file ... OK
writing chr2 sequence to file ... OK
writing chr3 sequence to file ... OK
writing chr4 sequence to file ... OK
writing chr5 sequence to file ... OK
writing chr6 sequence to file ... OK
writing chr7 sequence to file ... OK
writing chr8 sequence to file ... OK
writing chr9 sequence to file ... OK
writing chr10 sequence to file ... OK
writing chr11 sequence to file ... OK
writing chr12 sequence to file ... OK
writing chr13 sequence to file ... OK
writing chr14 sequence to file ... OK
writing chr15 sequence to file ... OK
writing chr16 sequence to file ... OK
writing chr17 sequence to file ... OK
writing chr18 sequence to file ... OK
writing chr19 sequence to file ... OK
writing chrX sequence to file ... OK
writing chrY sequence to file ... OK
writing chrM sequence to file ... OK
writing chr1_random sequence to file ... OK
writing chr3_random sequence to file ... OK
writing chr4_random sequence to file ... OK
writing chr5_random sequence to file ... OK


## Chicken
galGal5 for the big study

In [39]:
BiocManager::install("BSgenome.Ggallus.UCSC.galGal5")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.12), R 3.6.1 (2019-07-05)

Installing package(s) 'BSgenome.Ggallus.UCSC.galGal5'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocManager', 'devtools', 'dplyr', 'IRkernel', 'stringi'



In [7]:
library(BSgenome.Ggallus.UCSC.galGal5)

In [8]:
genome <- get("Ggallus")

In [9]:
dir.create("../../resources/WGBS_public/Gallus_gallus/refgenome")

In [10]:
export(genome, "../../resources/WGBS_public/Gallus_gallus/refgenome/galGal5.fasta")

writing chr1 sequence to file ... OK
writing chr2 sequence to file ... OK
writing chr3 sequence to file ... OK
writing chr4 sequence to file ... OK
writing chr5 sequence to file ... OK
writing chr6 sequence to file ... OK
writing chr7 sequence to file ... OK
writing chr8 sequence to file ... OK
writing chr9 sequence to file ... OK
writing chr10 sequence to file ... OK
writing chr11 sequence to file ... OK
writing chr12 sequence to file ... OK
writing chr13 sequence to file ... OK
writing chr14 sequence to file ... OK
writing chr15 sequence to file ... OK
writing chr16 sequence to file ... OK
writing chr17 sequence to file ... OK
writing chr18 sequence to file ... OK
writing chr19 sequence to file ... OK
writing chr20 sequence to file ... OK
writing chr21 sequence to file ... OK
writing chr22 sequence to file ... OK
writing chr23 sequence to file ... OK
writing chr24 sequence to file ... OK
writing chr25 sequence to file ... OK
writing chr26 sequence to file ... OK
writing chr27 sequenc

In [40]:
BiocManager::install("BSgenome.Ggallus.UCSC.galGal4")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.12), R 3.6.1 (2019-07-05)

Installing package(s) 'BSgenome.Ggallus.UCSC.galGal4'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocManager', 'devtools', 'dplyr', 'IRkernel', 'stringi'



## Koala:
Download and forge to BSgenome
downloading the assembly we need from https://www.ncbi.nlm.nih.gov/assembly/GCF_002099425.1/ and copy, then untar in the corresponding folder

Forging into a BSGenome package:

In [32]:
 ###where to save the output
seed_dir = file.path("/nobackup/lab_bock/projects/DNAmeth500species//resources/WGBS_public/Phascolarctos_cinereus/refgenome/")
seed_file = file.path(seed_dir, "phaCin_unsw_v4.1_seed.txt")

In [33]:
cat(c("Package: BSgenome.Pcinereus.NCBI.phaCinunswv41",
             "Title: Full genome sequences for Koala (NCBI assembly version phaCin_unsw_v4.1)",
             "Description: Full genome sequences for Phascolarctos cinereus (Koala), as provided by NCBI (phaCin_unsw_v4.1) and stored in Biostrings objects.",
             "Version: 1.1.1",
             "organism: Phascolarctos cinereus", 
            "common_name: Koala",
            "provider: NCBI",
            "provider_version: 4.1",
            "release_date:2017/04/18 ",
            "release_name: phaCin_unsw_v4.1",
            "source_url:  https://www.ncbi.nlm.nih.gov/assembly/GCF_002099425.1/ ",
             "organism_biocview: Phascolarctoscinereus",
            "BSgenomeObjname: Pcinereus",
            paste(c("seqs_srcdir: ", seed_dir),collapse=""),
            "seqfile_name: phaCin_unsw_v4.1.2bit"),sep = "\n",
                        file = seed_file)

In [19]:
library(BSgenome.Pcinereus.NCBI.phaCinunswv41)

## Cow

In [9]:
av_gen <- available.genomes(splitNameParts=TRUE)

Bioconductor version '3.10' is out-of-date; the current release version '3.12'
  is available with R version '4.0'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org




In [11]:
av_gen[av_gen$organism == "Btaurus",]

,pkgname,organism,provider,provider_version,masked
,<chr>,<fct>,<fct>,<chr>,<lgl>
8,BSgenome.Btaurus.UCSC.bosTau3,Btaurus,UCSC,bosTau3,FALSE
9,BSgenome.Btaurus.UCSC.bosTau3.masked,Btaurus,UCSC,bosTau3,TRUE
10,BSgenome.Btaurus.UCSC.bosTau4,Btaurus,UCSC,bosTau4,FALSE
11,BSgenome.Btaurus.UCSC.bosTau4.masked,Btaurus,UCSC,bosTau4,TRUE
12,BSgenome.Btaurus.UCSC.bosTau6,Btaurus,UCSC,bosTau6,FALSE
13,BSgenome.Btaurus.UCSC.bosTau6.masked,Btaurus,UCSC,bosTau6,TRUE
14,BSgenome.Btaurus.UCSC.bosTau8,Btaurus,UCSC,bosTau8,FALSE
15,BSgenome.Btaurus.UCSC.bosTau9,Btaurus,UCSC,bosTau9,FALSE


In [12]:
BiocManager::install("BSgenome.Btaurus.UCSC.bosTau9")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.12), R 3.6.1 (2019-07-05)

Installing package(s) 'BSgenome.Btaurus.UCSC.bosTau9'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocManager', 'devtools', 'dplyr', 'IRkernel', 'stringi'



In [13]:
library(BSgenome.Btaurus.UCSC.bosTau9)

In [14]:
dir.create("../../resources/WGBS_public/Bos_taurus/refgenome")

In [16]:
export(Btaurus, "../../resources/WGBS_public/Bos_taurus/refgenome/bosTau9.fasta")

writing chr1 sequence to file ... OK
writing chr2 sequence to file ... OK
writing chr3 sequence to file ... OK
writing chr4 sequence to file ... OK
writing chr5 sequence to file ... OK
writing chr6 sequence to file ... OK
writing chr7 sequence to file ... OK
writing chr8 sequence to file ... OK
writing chr9 sequence to file ... OK
writing chr10 sequence to file ... OK
writing chr11 sequence to file ... OK
writing chr12 sequence to file ... OK
writing chr13 sequence to file ... OK
writing chr14 sequence to file ... OK
writing chr15 sequence to file ... OK
writing chr16 sequence to file ... OK
writing chr17 sequence to file ... OK
writing chr18 sequence to file ... OK
writing chr19 sequence to file ... OK
writing chr20 sequence to file ... OK
writing chr21 sequence to file ... OK
writing chr22 sequence to file ... OK
writing chr23 sequence to file ... OK
writing chr24 sequence to file ... OK
writing chr25 sequence to file ... OK
writing chr26 sequence to file ... OK
writing chr27 sequenc

## Zebrafish

In [11]:
av_gen[av_gen$organism == "Drerio",]

,pkgname,organism,provider,provider_version,masked
,<chr>,<fct>,<fct>,<chr>,<lgl>
31,BSgenome.Drerio.UCSC.danRer10,Drerio,UCSC,danRer10,FALSE
32,BSgenome.Drerio.UCSC.danRer11,Drerio,UCSC,danRer11,FALSE
33,BSgenome.Drerio.UCSC.danRer5,Drerio,UCSC,danRer5,FALSE
34,BSgenome.Drerio.UCSC.danRer5.masked,Drerio,UCSC,danRer5,TRUE
35,BSgenome.Drerio.UCSC.danRer6,Drerio,UCSC,danRer6,FALSE
36,BSgenome.Drerio.UCSC.danRer6.masked,Drerio,UCSC,danRer6,TRUE
37,BSgenome.Drerio.UCSC.danRer7,Drerio,UCSC,danRer7,FALSE
38,BSgenome.Drerio.UCSC.danRer7.masked,Drerio,UCSC,danRer7,TRUE


In [12]:
BiocManager::install("BSgenome.Drerio.UCSC.danRer11")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.15), R 3.6.3 (2020-02-29)

Installing package(s) 'BiocVersion', 'BSgenome.Drerio.UCSC.danRer11'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocParallel', 'caret', 'cluster', 'DelayedArray',
  'GenomeInfoDb', 'GenomicAlignments', 'ggtree', 'IRanges', 'KernSmooth',
  'Matrix', 'mgcv', 'pillar', 'Rhtslib', 'ROCR', 'Rsamtools', 'S4Vectors',
  'simpleCache', 'stringi', 'SummarizedExperiment', 'tibble'



In [13]:
BiocManager::install("BSgenome.Drerio.UCSC.danRer10")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.15), R 3.6.3 (2020-02-29)

Installing package(s) 'BSgenome.Drerio.UCSC.danRer10'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'BiocParallel', 'caret', 'cluster', 'DelayedArray',
  'GenomeInfoDb', 'GenomicAlignments', 'ggtree', 'IRanges', 'KernSmooth',
  'Matrix', 'mgcv', 'pillar', 'Rhtslib', 'ROCR', 'Rsamtools', 'S4Vectors',
  'simpleCache', 'stringi', 'SummarizedExperiment', 'tibble'



## Lancelet:
The genome is predownloaded from http://amphiencode.github.io/Data/  (https://www.dropbox.com/s/r9s9wd3osyf6zud/Bl71nemr.fa.gz?dl=1)

In [16]:
getwd()

[1] "/binfl/lv71484/droman/DNAmeth500species/resources/WGBS_public"

In [20]:
genome_fa <- readDNAStringSet("Branchiostoma_lanceolatum/refgenome/Bl71nemr_nameswithsize.fa")

In [21]:
length(genome_fa)

[1] 10247

In [22]:
strsplit(names(genome_fa)[[1]], " ")[[1]][1]

[1] "Sc0000000"

In [23]:
names_nosize <- strsplit(names(genome_fa), " ")

In [24]:
names_nosize[[10246]]

[1] "xpSc0041287"                                  
[2] "old_C10706680;size:1641;start:1126;length:515"

In [25]:
names_clean <- sapply(names_nosize, function(x) x[1])

In [26]:
names(genome_fa) <- names_clean

In [27]:
head(genome_fa)

  A DNAStringSet instance of length 6
       width seq                                            names               
[1] 10571537 CTGGTCATGTGTTAACTTAGTA...TTATACCATCAAGGCAACAAA Sc0000000
[2]  9833161 CCTAACCCCAACCCCAACCCCA...AACAACTGCCTCAACCCGTAC Sc0000001
[3]  8797521 CGTCAGAGTTCGAACGAGATAG...GTTAAGCTTGCGAAGAATCGA Sc0000002
[4]  8740412 CCTAACCCTACAAACCATCTAA...CAAATACAAGGGTAGCAATAA Sc0000003
[5]  8020686 CCGAAAATTTTCCCCATAGGCA...CCACAAGTACAGTTCAGCCGG Sc0000004
[6]  7953648 TGACTTTGTTTACAATGAACTT...TTAGGGTTAGGGGTTAGGGGT Sc0000005

In [ ]:
## move the original file

In [28]:
writeXStringSet(genome_fa,"Branchiostoma_lanceolatum/refgenome/Bl71nemr.fa")

## Xenopus

In [ ]:
mkdir("Xenopus_laevis/refgenome")

In [ ]:
## koala

In [6]:
genome_fa <- readDNAStringSet("Phascolarctos_cinereus/refgenome/phaCin_unsw_v4.1.fa")
head(genome_fa)

  A DNAStringSet instance of length 6
       width seq                                            names               
[1] 40558015 ACAATGATCCAAGACAAGTACA...CCCCTTCCTGGCCACTCCTCT NW_018343952.1
[2] 39914628 CAAAAACAAAAAATTAAAAAAA...CCAGCTAAAATACCAAGCTTT NW_018343953.1
[3] 38256937 GAAGCAGAAGTCTAAAAATATT...CCTGTTCTATTGTCCCTTTGA NW_018343954.1
[4] 36543856 AATGAGTGGGGTTGGGACCAGG...TCAGCAGGCATAATATGTAAT NW_018343955.1
[5] 36077823 GAAGCCAGGCTAACTACCCTCC...GGAGGAGCTGCAGGAGTGGCC NW_018343956.1
[6] 29993169 AGGAATGCAAATTAAAACAACC...TTGAAGTAAAAAAAAAAAAAA NW_018343957.1

In [7]:
names_nosize <- strsplit(names(genome_fa), " ")

In [8]:
names_clean <- sapply(names_nosize, function(x) x[1])

In [9]:
names(genome_fa) <- names_clean

In [10]:
head(genome_fa)

  A DNAStringSet instance of length 6
       width seq                                            names               
[1] 40558015 ACAATGATCCAAGACAAGTACA...CCCCTTCCTGGCCACTCCTCT NW_018343952.1
[2] 39914628 CAAAAACAAAAAATTAAAAAAA...CCAGCTAAAATACCAAGCTTT NW_018343953.1
[3] 38256937 GAAGCAGAAGTCTAAAAATATT...CCTGTTCTATTGTCCCTTTGA NW_018343954.1
[4] 36543856 AATGAGTGGGGTTGGGACCAGG...TCAGCAGGCATAATATGTAAT NW_018343955.1
[5] 36077823 GAAGCCAGGCTAACTACCCTCC...GGAGGAGCTGCAGGAGTGGCC NW_018343956.1
[6] 29993169 AGGAATGCAAATTAAAACAACC...TTGAAGTAAAAAAAAAAAAAA NW_018343957.1

In [ ]:
file.copy(from = "Phascolarctos_cinereus/refgenome/phaCin_unsw_v4.1.fa", to="Phascolarctos_cinereus/refgenome/phaCin_unsw_v4.1_longnames.fa")

In [19]:
writeXStringSet(genome_fa,"Phascolarctos_cinereus/refgenome/phaCin_unsw_v4.1.fa")